<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import urllib.parse
import urllib.request
import xml.etree.ElementTree as ET
import io
from bs4 import BeautifulSoup
import time
import sys

In [6]:
onematch = pd.read_parquet('data/product_info140000.parquet')
onematch['categoryname']

50              Sparkling Wine
143                   Red Wine
198                   Red Wine
254                   Red Wine
334                      Cider
994                   Red Wine
996                   Red Wine
1039                  Red Wine
1079                  Red Wine
1170                White Wine
1182                          
1185                          
1235                White Wine
1390                     Vodka
1493                    Hybrid
1543                  Red Wine
1632    Gift and Sampler Packs
1697                     Vodka
1868            Whisky/Whiskey
1935                White Wine
2260                       Ale
2277                          
2366                  Red Wine
2508                White Wine
2601                  Red Wine
2620                  Red Wine
2643                White Wine
3388                  Red Wine
3438                       Rum
3793                  Red Wine
                 ...          
5996                  Red Wine
6010    

In [ ]:
onematch = pd.read_parquet('onematch.parquet')
product_ids = pd.read_parquet('product_ids.parquet')

In [ ]:
product_info10000 = pd.read_parquet('product_info10000.parquet')

In [ ]:
product_info10000.dropna().shape

Create a list of product ids we want to look up. We can then fill in all of the information about these product ids in a new table.

In [ ]:
newIds = (pd.DataFrame(product_ids.ProductID.append(onematch.ProductID).unique(),columns=['ProductID'])
          .sort_values('ProductID')
          .reset_index(drop=True)
         )
newIds['Response'] = None
newIds['XML'] = None
newIds.head()

Make a list of ids to pull

In [ ]:
newIds = list(product_ids.ProductID.append(onematch.ProductID).unique())
newIds.sort()

In [ ]:
newIds[1:10]

In [ ]:
product_info = scrape_all_info(newIds)

In [ ]:
range(10,20)

In [ ]:
product_info100000 = scrape_all_info2(range(100000))
product_info100000.to_parquet('product_info100000.parquet')

In [ ]:
range(1,10)

In [ ]:
for i in range(7,50): # 5,110
    maxrange = i * 10000
    minrange = (i-1) * 10000 + 1
    print('--')
    print(str(minrange) + ' to ' + str(maxrange))
    product_info_current = scrape_all_info2(range(minrange,maxrange))
    product_info_current.to_parquet('product_info'+str(maxrange)+'.parquet')
    

In [ ]:
for i in range(5,110):
    maxrange = i * 10000
    minrange = (i-1) * 10000 + 1
    print('--')
    print(str(minrange) + ' to ' + str(maxrange))
    #product_info_current = scrape_all_info2(range(minrange,maxrange))
    #product_info_current.to_parquet('product_info'+str(maxrange)+'.parquet')
    

In [ ]:
product_info100000.shape()

In [ ]:
def scrape_all_info2(idlist):
    fieldlist = ['language', 'itemnumber', 'itemname', 'price', 'productsize', 'isdiscontinued', 'islimited',
                 'am', 'ambonusmiles', 'amexpiration', 'kosher', 
                 'lto', 'ltoexpiration', 'ltoregularprice', 'ltosavings', 
                 'releasedate', 'upcnumber', 'sccnumber', 'vao', 'vaodescription', 'vaoexpiration', 
                 'vqa', 'productquantity', 'producer', 'producingsubregion', 'producingregion', 'producingcountry', 
                 'alcoholpercentage', 'sugarcontent', 'tastingnotes', 'pairings', 
                 'itemdescription', 'liquortype', 'categoryname', 'stocktype', 'subcategoryname',
                 'sellingpackage', 'sweetnessdescriptor', 'winestyle', 'winevarietal', 
                 'stylebody', 'stylebody_fr', 'styleflavour', 'styleflavour_fr']

    productinfo = pd.DataFrame(columns=fieldlist)
    
    #timer/counter
    start_time = int(time.time())
    count = 0
    total = len(idlist)
    

    for id in idlist:
        #print(id)
        # Pull xml from lcbo api
        xmltest = pullXML(str(id))
        # Extract dict of values
        newitem = parse_item_xml(xmltest, fieldlist)

        # Add to our dataframe
        productinfo = productinfo.append(newitem,ignore_index=True)
        
        #counter/timer
        end_time = int(time.time())
        min_spent = round(((end_time-start_time)/60),2)
        remaining_time = round((min_spent/max(count,1))*max((total-count),1),2)

        count_str = 'extracted ' + str(count) + '/' + str(total) + \
                ' products in total ' + str(min_spent) + ' minutes' + \
                ' estimated time remaining is ' + str(remaining_time) + ' minutes'
        if count % 100 == 0:
            sys.stdout.write("\r{}".format(count_str))
        count += 1
        
        # sleep so we don't get blacklisted
        #time.sleep(0.5)
        
    #save to parquet, also return the pandas dataframe
    #productinfo.to_parquet("productinfo2.parquet")
    return productinfo

In [ ]:
def scrape_all_info(idlist):
    fieldlist = ['language', 'itemnumber', 'itemname', 'price', 'productsize', 'isdiscontinued', 'islimited',
                 'am', 'ambonusmiles', 'amexpiration', 'kosher', 
                 'lto', 'ltoexpiration', 'ltoregularprice', 'ltosavings', 
                 'releasedate', 'upcnumber', 'sccnumber', 'vao', 'vaodescription', 'vaoexpiration', 
                 'vqa', 'productquantity', 'producer', 'producingsubregion', 'producingregion', 'producingcountry', 
                 'alcoholpercentage', 'sugarcontent', 'tastingnotes', 'pairings', 
                 'itemdescription', 'liquortype', 'categoryname', 'stocktype', 'subcategoryname',
                 'sellingpackage', 'sweetnessdescriptor', 'winestyle', 'winevarietal', 
                 'stylebody', 'stylebody_fr', 'styleflavour', 'styleflavour_fr']

    productinfo = pd.DataFrame(columns=fieldlist)
    
    #timer/counter
    start_time = int(time.time())
    count = 0
    total = len(idlist)

    for id in idlist:
        print(id)
        # Pull xml from lcbo api
        xmltest = pullXML(id)
        # Extract dict of values
        newitem = parse_item_xml(xmltest, fieldlist)

        # Add to our dataframe
        productinfo = productinfo.append(newitem,ignore_index=True)
        
        #counter/timer
        end_time = int(time.time())
        min_spent = round(((end_time-start_time)/60),2)
        remaining_time = round((min_spent/max(count,1))*max((total-count),1),2)

        count_str = 'extracted ' + str(count) + '/' + str(total) + \
                ' reviews in total ' + str(min_spent) + ' minutes' + \
                ' estimated time remaining is ' + str(remaining_time) + ' minutes'
        sys.stdout.write("\r{}".format(count_str))
        count += 1
        
        # sleep so we don't get blacklisted
        time.sleep(3)
        
    #save to parquet, also return the pandas dataframe
    productinfo.to_parquet("productinfo.parquet")
    return productinfo

In [ ]:
def parse_item_xml(xmlstring, fieldlist):
    soup = BeautifulSoup(xmlstring)
    product = soup.find('products')
    
    newitem = {}
    
    for field in fieldlist:
        value = product.find(field)
        if value is not None:
            newitem[field] = value.text

    return newitem

In [ ]:
def pullXML(ProductID):
    url = 'http://www.foodanddrink.ca/lcbo-webapp/productdetail.do?itemNumber=' + ProductID
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'language' : 'en' }
    headers = { 'User-Agent' : user_agent }
    data = urllib.parse.urlencode(values)
    data = data.encode('ascii')
    req = urllib.request.Request(url, data, headers)
    try:
        with urllib.request.urlopen(req) as response:
           content = response.read()
        return content
        #return BeautifulSoup(content, 'xml')
    except :
        raise response
        